In [424]:
import boto3
from file_extraction import *
from to_dataframe import *
from clean_talent_json import *
import pandas as pd
import datetime, json

In [425]:
s3_client = boto3.client('s3')
files_dict = {'academy_csv': [], 'json': [], 'txt': [], 'csv': []}

In [426]:
# extract_file_type(s3_client, 'Talent', files_dict, 'txt')
# applicants = files_dict['txt']

# txt_df = convert_all_to_df(applicants, 'txt')

In [427]:
# extract_file_type(s3_client, 'Talent', files_dict, 'json')
# applicants = files_dict['json']

# json_df = convert_all_to_df(applicants, 'json')

# json_df.to_csv('talent_json_all.csv', index=False)

In [428]:
# json_file = s3_client.get_object(Bucket='data-eng-228-final-project', Key='Talent/10401.json')
# json_df = json_to_pd(json_file)
# json_df


In [429]:
# extract_file_type(s3_client, 'Talent', files_dict, 'csv')
# applicants = files_dict['csv']

# csv_df = convert_all_to_df(applicants, 'csv')


In [430]:
# duplicated_names = csv_df[csv_df.duplicated(subset='name', keep=False)]
# print(duplicated_names)

In [431]:
# extract_file_type(s3_client, 'Academy', files_dict, 'csv')
# applicants = files_dict['academy_csv']

# ac_csv_df = convert_all_to_df(applicants, 'csv')
# ac_csv_df

In [432]:
# duplicated_names = ac_csv_df[ac_csv_df.duplicated(subset='name', keep=False)]
# print(duplicated_names)

In [433]:
# Amy's code for courses table
def create_courses(dataframe):
    courses_df  = dataframe
    courses_df = pd.DataFrame().assign(course = courses_df['course_name'], date = courses_df['course_date'])
    courses_df['course_id'] = courses_df.groupby(['course']).ngroup()
    courses_df = courses_df[['course', 'course_id', 'date']].copy()
    courses_df.set_index('course_id', inplace=True)
    return courses_df


In [437]:
def academy_table():
    # Initialize an empty list to store data
    course_table = []

    # List the objects in the S3 bucket
    response = s3_client.list_objects(Bucket="data-eng-228-final-project", Prefix="Academy")

    # Iterate through the objects in the S3 bucket
    for obj in response['Contents']:
        file_key = obj['Key']
        
        # Extract the course name from the file key
        file_name = file_key.split('/')[-1].split('.')[0].split('_')[:2]
        course_name = file_name[0] + '_' + file_name[1]
        course_group = file_name[0]
        course_date = file_key.split('/')[-1].split('.')[0].split('_')[-1:]
        
        # Read the CSV file from S3 and extract student IDs
        csv_object = s3_client.get_object(Bucket="data-eng-228-final-project", Key=file_key)
        df = pd.read_csv(csv_object['Body'])
        student_ids = df['name'].tolist()
        
        # Append the data to the list
        course_table.extend([(student_id, course_name, course_group, course_date[0]) for student_id in student_ids])

    # Create the DataFrame
    course_df = pd.DataFrame(course_table, columns=['name', 'course_name', 'course_group','course_date'])
    
    # Return the DataFrame
    return course_df

test = academy_table()
test[test['name'].duplicated()]

,name,course_name,course_group,course_date


In [438]:
def course_table(academy_table):
    course_df = academy_table['course_name'].unique()
    course_df = pd.DataFrame(course_df, columns=['course_name'])
    course_df = course_df.rename_axis('course_id')
    return course_df

course_table(academy_table())


,course_name
course_id,
0,Business_20
1,Business_21
2,Business_22
3,Business_23
4,Business_24
5,Business_25
6,Business_26
7,Business_27
8,Business_28


In [439]:
academy_df = academy_table()
academy_df['course_date'] = pd.to_datetime(academy_df['course_date'], dayfirst=True, format='ISO8601')

candidates_df = pd.read_csv("talent_json_all.csv")
candidates_df['date'] = candidates_df['date'].str.replace('//', '/')
candidates_df['date'] = pd.to_datetime(candidates_df['date'], dayfirst=True)

course_df = course_table(academy_table())


def academy_table_df(course_table, academy_table, candidates):
    academy = []

    for index, row in candidates.iterrows():
        # Assigning variables to values in candidate table
        inv_date = row['date']
        interest = row['course_interest']
        # 'name' to be changed to 'name_iD' from the candidates table
        candidate_id = row['name']
        
        if row['result'] == 'Pass':
            # Keeping only the group that is candidates 'interest'
            grouped_courses = academy_table.groupby('course_group').get_group(interest)
            # Keeping only groups that has start dates after candidates interview
            start_date = grouped_courses[grouped_courses['course_date'] > inv_date]
            # Taking the first group after sorting them in ascending order
            candidate_course = start_date.sort_values('course_date', ascending=True).iloc[0, 1]
            # Adding candidates_id and course_name to the list
            academy.append([candidate_course, candidate_id])
            academy_df = pd.DataFrame(academy, columns=['course_name', 'name_id'])
            
    course_table['course_id'] = course_table.index
    merged_df = pd.merge(course_table, academy_df,  on='course_name', how='inner')
    merged_df = merged_df.set_index('course_id')
    academy_df = pd.DataFrame(merged_df['name_id'])
    
    return academy_df

test_academy = academy_table_df(course_df, academy_df, candidates_df)
test_academy

test_academy[test_academy['name_id'] == 'Thom Derwin']
    

,name_id
course_id,
35,Thom Derwin
